## Final Project Submission

Please fill out:
* Student name: Nick Kolowich
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: Rafael Carrasco
* Blog post URL:


In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib qt

#pandas and numpy
import pandas as pd
import numpy as np

# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# sk learn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE 

# stats and scipy
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.formula.api import ols
import scipy.stats as stats


#pd.options.display.float_format = '{:,.4f}'.format

#### Import and Investigate

In [34]:
data = pd.read_csv(r'C:\Users\Nick\Documents\Flatiron Project 2\dsc-phase-2-project-online-master\dsc-phase-2-project-online-master\data\kc_house_data.csv')

FileNotFoundError: [Errno 2] File ‪C:\Users\Nick\Downloads\league_na1.csv does not exist: '\u202aC:\\Users\\Nick\\Downloads\\league_na1.csv'

In [3]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [5]:
data.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [6]:
#cleans the sqft basement column, removing "?" and replacing with 0's

data['sqft_basement'] = np.where(data['sqft_living'] == data['sqft_above'], 0, data['sqft_basement'])

#calculates missing basement values by subtracting sqft_above from sqft_living

data['sqft_basement'] = np.where(data['sqft_living'] >= data['sqft_above'], data['sqft_living']-data['sqft_above'], data['sqft_basement'])

# removes row with 33 bedrooms

data = data[data['bedrooms'] != 33]

# saves cleaned dataframe to a csv

In [7]:
data_ = data[['price','bedrooms','bathrooms','sqft_living','sqft_lot','floors','condition','grade','sqft_above','sqft_basement','yr_built','yr_renovated','sqft_living15','sqft_lot15']].replace(0, np.nan)
data_.columns = ['Price','Bedrooms','Bathrooms','sqft_House','sqft_Lot','Floors','Condition','Grade','sqft_Above_Ground','sqft_Basement','Year_Built','Year_Renovated','sqft_Nearby_Homes','sqft_Nearby_Lots']

# converts cleaned dataframe to z-scores

data_z = (data_ - data_.mean())/data_.std()

In [8]:
# renames data_z columns

data_z.columns = ['Price','Bedrooms','Bathrooms','sqft_House','sqft_Lot','Floors','Condition','Grade','sqft_Above_Ground','sqft_Basement','Year_Built','Year_Renovated','sqft_Nearby_Homes','sqft_Nearby_Lots']

# builds a correlation matrix for the z-score columns

correlation = data_z.corr()

# displays 90th percentile correlation values

correlation.describe(percentiles=[0.9])

,Price,Bedrooms,Bathrooms,sqft_House,sqft_Lot,Floors,Condition,Grade,sqft_Above_Ground,sqft_Basement,Year_Built,Year_Renovated,sqft_Nearby_Homes,sqft_Nearby_Lots
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000
mean,0.389631,0.339129,0.461307,0.519074,0.189977,0.265517,-0.025793,0.467252,0.482621,0.338633,0.267964,0.138080,0.436896,0.196930
std,0.299457,0.271321,0.299096,0.327220,0.298639,0.325732,0.331198,0.314634,0.328029,0.287464,0.317309,0.296621,0.310062,0.296732
min,0.036031,0.023440,-0.126443,-0.059397,-0.082031,-0.264013,-0.362244,-0.146854,-0.158826,-0.112100,-0.361555,-0.362244,-0.092979,-0.066791
50%,0.361665,0.341067,0.516540,0.647554,0.102303,0.268446,-0.076188,0.453318,0.508257,0.378901,0.283708,0.129044,0.438983,0.104636
90%,0.691739,0.573586,0.735028,0.842347,0.557982,0.517552,0.125357,0.760764,0.840335,0.644136,0.501768,0.252097,0.749007,0.561263
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [33]:
scatter_matrix = pd.plotting.scatter_matrix(data_[significant_features], figsize=[20, 20]);

In [31]:
# scatterplot of the two variables with linear regression

fig = plt.figure(figsize=(13,8))
sns.regplot(x=data['zipcode'], y=data_['Price'], marker='o', color='dodgerblue', scatter_kws={'s':1.5}, line_kws={"color": "mediumspringgreen"})

# sets the labels and titles of the plot

plt.title('Price vs. Zipcode', y=1.0, fontsize =16)
plt.xlabel('Zip Code', fontsize=12) 
plt.ylabel('Price', fontsize=12)

# calculates the linear regression values using scipy's linregress function

slope, intercept, r_value, p_value, std_err = stats.linregress(data['zipcode'], [data_['Price']])

# squares the r-value from the linear regression to find R-squared

plt.text(98005, 7750000, 'R-squared:' + str(round((r_value**2),4)) , fontsize=12);

print(slope,intercept, r_value)

-366.65271278464127 36500838.6398792 -0.05340867902083491


In [26]:
# scatterplot of the two variables with linear regression

fig = plt.figure(figsize=(13,8))
sns.regplot(x=data_['Grade'], y=data_['Price'], marker='o', color='dodgerblue', scatter_kws={'s':1.5}, line_kws={"color": "mediumspringgreen"})

# sets the labels and titles of the plot

plt.title('Price vs. Grade', y=1.0, fontsize =16)
plt.xlabel('Grade', fontsize=12) 
plt.ylabel('Price', fontsize=12)

# calculates the linear regression values using scipy's linregress function

slope, intercept, r_value, p_value, std_err = stats.linregress(data['zipcode'], [data_['Price']])

# squares the r-value from the linear regression to find R-squared

plt.text(98000, 7750000, 'R-squared:' + str(round((r_value**2),4)) , fontsize=12);

print(slope)

209163.02896126156


In [10]:
# creates the figure and axis for the subplots

fig, ax = plt.subplots(figsize=(13, 8))

# creates a mask to remove the mirrored half of heatmap

mask = np.triu(np.ones_like(correlation, dtype=np.bool))

# adjusts mask and dataframe

mask_adj = mask[1:, :-1]
correlation_adj = correlation.iloc[1:,:-1].copy()

# plots heatmap

sns.heatmap(correlation_adj, mask=mask_adj, annot=True, fmt='.2f', cmap='Blues', linewidths=3, vmin=-0.5, vmax=0.9, cbar_kws={"shrink": .8})

# ytick adjustment

plt.yticks(rotation=0)
plt.show()

In [11]:
# Defining the problem
target = 'Price'
features = ['Bedrooms','Bathrooms','sqft_House','sqft_Lot','Floors','Condition','Grade','sqft_Above_Ground','sqft_Basement','Year_Built','Year_Renovated','sqft_Nearby_Homes','sqft_Nearby_Lots']

predictors = '+'.join(features)
formula = target + '~' + predictors
model = ols(formula=formula, data=data_z).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.768
Method:                 Least Squares   F-statistic:                     106.1
Date:                Fri, 09 Oct 2020   Prob (F-statistic):          1.36e-111
Time:                        14:12:32   Log-Likelihood:                -527.73
No. Observations:                 382   AIC:                             1081.
Df Residuals:                     369   BIC:                             1133.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.0009      0.108     -0.009      0.993      -0.213       0.211
Bedrooms             -0.1577      0.049     -3.197      0.002      -0.255      -0.061
Bathrooms             0.0478      0.073      0.650      0.516      -0.097       0.192
sqft_House            0.5753      0.047     12.218      0.000       0.483       0.668
sqft_Lot             -0.1395      0.090     -1.550      0.122      -0.317       0.037
Floors               -0.2774      0.083     -3.349      0.001      -0.440      -0.114
Condition             0.0656      0.071      0.920      0.358      -0.075       0.206
Grade                 0.3868      0.079      4.906      0.000       0.232       0.542
sqft_Above_Ground     0.5804      0.060      9.658      0.000       0.462       0.699
sqft_Basement         0.1189      0.060      1.968      0.050       0.000       0.238
Year_Built           -0.3582      0.078     -4.604      0.000      -0.511      -0.205
Year_Renovated        0.0717      0.065      1.097      0.273      -0.057       0.200
sqft_Nearby_Homes     0.3241      0.065      4.954      0.000       0.195       0.453
sqft_Nearby_Lots     -0.0627      0.089     -0.706      0.481      -0.237       0.112
==============================================================================
Omnibus:                      100.617   Durbin-Watson:                   1.933
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              736.957
Skew:                           0.892   Prob(JB):                    9.37e-161
Kurtosis:                       9.566   Cond. No.                     9.28e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.22e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [12]:
#building an OLS model with significant features (p<0.05)
target = 'Price'
significant_features = ['Bedrooms', 'sqft_House', 'Floors', 'Grade','sqft_Above_Ground','sqft_Nearby_Homes']
significant_features_orig = ['']
significant_predictors = '+'.join(significant_features)
formula = target + '~' + significant_predictors
model = ols(formula=formula, data=data_z).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.551
Method:                 Least Squares   F-statistic:                     4420.
Date:                Fri, 09 Oct 2020   Prob (F-statistic):               0.00
Time:                        14:12:32   Log-Likelihood:                -21991.
No. Observations:               21596   AIC:                         4.400e+04
Df Residuals:                   21589   BIC:                         4.405e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept          9.021e-17      0.005   1.98e-14      1.000      -0.009       0.009
Bedrooms             -0.1169      0.006    -20.342      0.000      -0.128      -0.106
sqft_House            0.6498      0.012     55.499      0.000       0.627       0.673
Floors               -0.0236      0.006     -4.109      0.000      -0.035      -0.012
Grade                 0.3263      0.008     40.874      0.000       0.311       0.342
sqft_Above_Ground    -0.1689      0.011    -15.189      0.000      -0.191      -0.147
sqft_Nearby_Homes     0.0384      0.008      5.081      0.000       0.024       0.053
==============================================================================
Omnibus:                    16627.022   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           961625.868
Skew:                           3.208   Prob(JB):                         0.00
Kurtosis:                      35.055   Cond. No.                         6.43
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
data_columns = data_z[significant_features]
vif = [variance_inflation_factor(data_columns.values, i) for i in range(data_columns.shape[1])]
list(zip(significant_features, vif))

[('Bedrooms', 1.5900096228382512),
 ('sqft_House', 6.594932213178756),
 ('Floors', 1.590282381761327),
 ('Grade', 3.066202112505796),
 ('sqft_Above_Ground', 5.951125700563694),
 ('sqft_Nearby_Homes', 2.7409824292487737)]

In [14]:
correlation_significant_features_df = data_z[significant_features]
correlation_significant_features_df.insert(loc=0, column='Price', value=data_z['Price'])
correlation_significant_features = correlation_significant_features_df.corr()

# creates the figure and axis for the subplots

fig1, ax1 = plt.subplots(figsize=(13, 8))

# creates a mask to remove the mirrored half of heatmap

mask = np.triu(np.ones_like(correlation_significant_features, dtype=np.bool))

# adjusts mask and dataframe

mask_sig_adj = mask[1:, :-1]
correlation_sig_adj = correlation_significant_features.iloc[1:,:-1].copy()

# plots heatmap

sns.heatmap(correlation_sig_adj, mask=mask_sig_adj, annot=True, fmt='.2f', cmap='Blues', linewidths=3, vmin=-0.5, vmax=0.9, cbar_kws={"shrink": .8})

# ytick adjustment

plt.yticks(rotation=0)
plt.show()

In [15]:
# create training and testing vars
data_drop_nulls = data_.replace(np.nan, 0)
X_train, X_test, y_train, y_test = train_test_split(data_drop_nulls, data_drop_nulls['Price'], test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17276, 14) (17276,)
(4320, 14) (4320,)


In [16]:
linreg = LinearRegression(fit_intercept=False)
linreg.fit(X_train, y_train)

LinearRegression(fit_intercept=False)

In [17]:
linreg.coef_

array([ 1.00000000e+00, -5.10673387e-12, -5.80086795e-11, -3.97403608e-16,
       -1.43321587e-17,  9.30062119e-11,  1.85359504e-11, -2.25564788e-11,
       -1.33115958e-15,  9.30962247e-16,  9.12452846e-16, -4.55465255e-16,
        2.70996591e-16,  1.71404425e-17])

The features with the highest correlation to price were

1. sqft. of House
2. Grade
3. sqft. Above Ground
4. sqft of Nearby Homes
5. Bathrooms



In [18]:
fig, ax = plt.subplots()
ax = fig.add_subplot(111, projection='3d')

x=data_['Grade']
y=data_['sqft_House']
z=data_['Price']

x = np.array(x)
y = np.array(y)
z = np.array(z)

ax.scatter(x, y, z, c='springgreen', marker='o',linewidths=1, edgecolors='blue', alpha=0.8)
fig.set_size_inches(13,8)
plt.title('Scatter of Price vs. Grade & Sqft. of House')
ax.set_xlabel('Grade')
ax.set_ylabel('sqft. of House')
ax.set_zlabel('Price')
ax.zaxis.set_tick_params(labelsize=8)

ax.ticklabel_format(axis='z', style='plain')

plt.show()

In [19]:
residplot = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)

In [20]:
plt.scatter(model.predict(data_[significant_features]), model.resid)
plt.plot(model.predict(data_[significant_features]), [0 for i in range(len(data_))]);

In [21]:
for i in range(90, 99):
    q = i / 100
    print('{} percentile: {}'.format(q, data_['Price'].quantile(q=q)))

0.9 percentile: 887000.0
0.91 percentile: 919994.5
0.92 percentile: 950000.0
0.93 percentile: 997967.5
0.94 percentile: 1060000.0
0.95 percentile: 1160000.0
0.96 percentile: 1260000.0
0.97 percentile: 1390000.0
0.98 percentile: 1600000.0


In [22]:
subset = data_[data_['Price'] < 1000000]
print('Percent removed:',(len(data_) - len(subset))/len(data_))
outcome = 'Price'
predictors = '+'.join(significant_features)
formula = outcome + '~' + predictors
model = ols(formula=formula, data=subset).fit()
model.summary()

Percent removed: 0.06899425819596222


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.458
Model:                            OLS   Adj. R-squared:                  0.458
Method:                 Least Squares   F-statistic:                     2829.
Date:                Fri, 09 Oct 2020   Prob (F-statistic):               0.00
Time:                        14:12:32   Log-Likelihood:            -2.6733e+05
No. Observations:               20106   AIC:                         5.347e+05
Df Residuals:                   20099   BIC:                         5.347e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -2.531e+05   9581.500    -26.418      0.000   -2.72e+05   -2.34e+05
Bedrooms          -1.199e+04   1467.422     -8.173      0.000   -1.49e+04   -9116.443
sqft_House          118.7525      3.160     37.577      0.000     112.558     124.947
Floors             1.404e+04   2402.196      5.846      0.000    9334.329    1.88e+04
Grade              6.944e+04   1577.184     44.030      0.000    6.64e+04    7.25e+04
sqft_Above_Ground   -61.3787      3.235    -18.971      0.000     -67.720     -55.037
sqft_Nearby_Homes    47.2221      2.704     17.467      0.000      41.923      52.521
==============================================================================
Omnibus:                      799.454   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              897.695
Skew:                           0.505   Prob(JB):                    1.17e-195
Kurtosis:                       3.226   Cond. No.                     3.24e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.24e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
fig_drop_outliers = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)

In [24]:
fig, ax = plt.subplots()
ax = fig.add_subplot(111, projection='3d')

x=subset['Grade']
y=subset['sqft_House']
z=subset['Price']

x = np.array(x)
y = np.array(y)
z = np.array(z)

ax.scatter(x, y, z, c='springgreen', marker='o',linewidths=1, edgecolors='blue', alpha=0.8)
fig.set_size_inches(13,8)
plt.title('Scatter of Price vs. Grade & Sqft. of House')
ax.set_xlabel('Grade')
ax.set_ylabel('sqft. of House')
ax.set_zlabel('Price')
ax.zaxis.set_tick_params(labelsize=8)

ax.ticklabel_format(axis='z', style='plain')

plt.show()